# Gerando Visualizações dos Dados

Ajuda Folium
https://python-visualization.github.io/folium/
    
Exemplos
http://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/

In [33]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import folium

In [34]:
#carrega todos mapas de datasets
rj = gpd.read_file('../Dados/Mapas/RJ-MUNIC/RJ-MUNIC.shp')
geo_dados = gpd.read_file('../Dados/Mapas/RJ-DATASET/RJ-DATASET-CENSO.shp')
setor = gpd.read_file('../Dados/Mapas/RJ-SETOR/RJ-SETOR-CENSO.shp')
bairro = gpd.read_file('../Dados/Mapas/RJ-BAIRROS/RJ-BAIRRO.shp')

In [35]:
#padrão de coordenadas que será utilizado
crs = {'init': 'epsg:4326'}

In [36]:
#evitar erros no gráfico para falta de dados
import numpy as np
setor = setor.fillna(np.nan)

In [37]:
#alterar já no geodataframe
rj.to_crs(crs, inplace = True)

In [38]:
#identificar o centro do mapa que será utilizado no openstreetmap, todo polígono possui um centro, para obter cada centro usar:
y = rj.centroid.y.iloc[0]

In [39]:
x = rj.centroid.x.iloc[0]

In [40]:
display(x, y)
display(rj)

-43.45025729553401

-22.92318738485854

,ID,CD_GEOCODM,NM_MUNICIP,geometry
0,1535,3304557,RIO DE JANEIRO,"MULTIPOLYGON (((-43.12376 -22.80712, -43.12380..."


# Mapa Camada Simples

In [42]:
#Inicializando o folium como o nosso centroid e usando a classe cloroplath atualizada
base = folium.Map([y, x], zoom_start=11, tiles='OpenStreetMap')
folium.Choropleth(
    geo_data=rj,
    fill_color='red',
    fill_opacity=0.3,
    line_weight=2,
).add_to(base)
base

In [43]:
#adicionar popup com nome do mapa
base = folium.Map([y, x], zoom_start=11, tiles='OpenStreetMap')
geojson_rj = folium.GeoJson(rj)
geojson_rj.add_child(folium.Popup(rj.NM_MUNICIP.iloc[0]))
geojson_rj.add_to(base)
base

# Mapa Várias Camadas

In [44]:
bairro[0:10]

,NM_BAIRRO,V002,geometry
0,Abolição,11356.0,"POLYGON ((674.902 7467.737, 674.936 7467.767, ..."
1,Acari,26967.0,"POLYGON ((670.474 7474.718, 670.405 7474.689, ..."
2,Alto da Boa Vista,9189.0,"POLYGON ((674.901 7457.693, 674.898 7457.693, ..."
3,Anchieta,55634.0,"POLYGON ((665.693 7474.737, 665.709 7474.722, ..."
4,Andaraí,39325.0,"POLYGON ((679.002 7462.700, 679.002 7462.700, ..."
5,Anil,24156.0,"POLYGON ((671.154 7460.208, 671.176 7460.202, ..."
6,Bancários,12512.0,"MULTIPOLYGON (((687.902 7477.410, 687.903 7477..."
7,Bangu,240666.0,"POLYGON ((653.298 7470.317, 653.272 7470.309, ..."
8,Barra da Tijuca,135920.0,"MULTIPOLYGON (((673.371 7451.196, 673.371 7451..."
9,Barra de Guaratiba,3545.0,"MULTIPOLYGON (((647.366 7446.582, 647.352 7446..."


In [45]:
bairro.to_crs(crs, inplace=True)

In [46]:
bairro[1:2]

,NM_BAIRRO,V002,geometry
1,Acari,26967.0,"POLYGON ((-43.33882 -22.82658, -43.33949 -22.8..."


In [48]:
base = folium.Map([y, x], zoom_start=11, tiles='OpenStreetMap')

for i in range(len(bairro)):
    geo = folium.GeoJson(
        bairro[i:i+1],
        name=bairro['NM_BAIRRO'][i],
        style_function=lambda feature: {
            'fillColor': 'green' if feature['properties']['V002'] > 50000 else 'yellow',
            'color': 'black',
            'weight': 1
        })
    label = '{} - {} habitantes'.format(bairro['NM_BAIRRO'][i], bairro['V002'][i])
    folium.Popup(label).add_to(geo)
    geo.add_to(base)
    
folium.LayerControl().add_to(base)

base.save('Bairros (RJ) gerado.html')

# Mapa com Marcadores

In [49]:
base = folium.Map([y, x], zoom_start=11, tiles='OpenStreetMap')

folium.Choropleth(
    geo_data=rj,
    name="Rio de Janeiro",
    line_color="Black",
    line_weight=3,
    fill_opacity=0,
).add_to(base)

#obtem uma amostra dos dados, não todos
amostra = geo_dados.sample(500)

#exemplos de ícones para usar nos marcadores
#https://fontawesome.com/icons?from=io

#iterar nas tuplas da amostra
for item in amostra.itertuples():
    #adiciona como filho na base um marcador, o marcador possui o ícone que foi usado a classe CSS do fontawesome
    base.add_child(folium.Marker(location=[item.Latitude, item.Longitude],
                                popup="<h4>"+str(item.Tipo)+"</h4> <h5>"+str(item.Bairro)+"</h5> <p>R$ "+str(item.Valor)+",00</p>",
                                icon=folium.Icon(color='red',
                                                 prefix='fa',
                                                 icon='fas fa-home')
                                )
                  )

folium.LayerControl().add_to(base)

base.save('Folium Marcadores gerado.html')

# Mapa com Marcadores Clusterizados

In [51]:
#plugin do Folium para clusterizar marcadores
from folium.plugins import MarkerCluster

In [53]:
base = folium.Map([y, x], zoom_start=11, tiles='OpenStreetMap')

folium.Choropleth(
    geo_data=rj,
    name="Rio de Janeiro",
    line_color="Black",
    line_weight=3,
    fill_opacity=0,
).add_to(base)

cluster = MarkerCluster(name='Imóveis Anunciados')

for item in geo_dados.itertuples():
    cluster.add_child(folium.Marker(location=[item.Latitude, item.Longitude],
                                popup="<h4>"+str(item.Tipo)+"</h4> <h5>"+str(item.Bairro)+"</h5> <p>R$ "+str(item.Valor)+",00</p>",
                                icon=folium.Icon(color='red',
                                                 prefix='fa',
                                                 icon='fas fa-home')
                                )
                  )
    
base.add_child(cluster)

folium.LayerControl().add_to(base)

base.save('Folium Marcadores Clusterizados.html')